## HW3

<b>In this homework, you will build an Jupyter notebook for predicting house prices, using data from King County, USA, the region where the city of Seattle is located.

This assignment covers three main concepts: 

<font color = 'blue'>1. Polynomial Regression 
<font color = 'blue'><br>2. Ridge Regression 
<font color = 'blue'><br>3. Lasso Regression

In [1]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv('home_data.csv', index_col = 0)

In [4]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### 1. Polynomial Regression

<b>A. Split the original data into train-validate set (90%) and test set (10%) using random state = 0.

In [5]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [6]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [7]:
X = df[['sqft_living']]
y = df.price

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

<b>B. Build a polynomial regression model with the degree = 15 using just ‘sqft_living’ on the train-validation set. Remember to run standard scaler to normalize the features before building your model.

In [9]:
pipe = make_pipeline(PolynomialFeatures(degree = 15), StandardScaler(), LinearRegression())

In [10]:
reg_poly = pipe.fit(X_train, y_train)

<b>C. Report the model’s performance on the train-validate set and the test set.

In [11]:
print("Test set score: {:.2f}".format(reg_poly.score(X_test, y_test)))

Test set score: 0.52


### 2. Ridge Regression

<b>A. Split the original data into train-validate set (90%) and test set (10%) using random state = 0.

In [12]:
X = df[['sqft_living']]
y = df.price

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

<b>B. For each L2_penalty λ in [10^3, 10^3.5, 10^4, 10^4.5, ..., 10^9], use GridSearchCV and 10-fold cross validation to compare the performance of the ridge regression with polynomial degree = 15 using just ‘sqft_living’ on the train-validation set. Remember to run standard scaler to normalize the features before building your model.

In [14]:
from sklearn.model_selection import GridSearchCV

<b>** Had to add the normalize = True in the Ridge() model - kept getting an error message.</b>

In [15]:
pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree = 15),
                     Ridge(normalize = True))

In [16]:
param_grid = {'ridge__alpha': np.logspace(3, 9, num = 13)}

In [17]:
grid = GridSearchCV(pipe, param_grid = param_grid, cv = 10, n_jobs = -1)

In [18]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('polynomialfeatures',
                                        PolynomialFeatures(degree=15,
                                                           include_bias=True,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('ridge',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=True, random_state=N...
      

In [19]:
print(grid.best_params_)
print(grid.best_score_)

{'ridge__alpha': 1000.0}
0.0009689423537825315


<b>** The score seems a bit off. Attempted with the `alpha` values below.

In [20]:
param_grid_test = {'ridge__alpha': (0.1, 0.11, 0.12, 0.15, 0.155, 0.2, 1)}

In [21]:
grid = GridSearchCV(pipe, param_grid = param_grid_test, cv = 10, n_jobs = -1)

In [22]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('polynomialfeatures',
                                        PolynomialFeatures(degree=15,
                                                           include_bias=True,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('ridge',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=True, random_state=None,
     

<b>C. Report which L2 penalty λ produced the lowest average validation error. Report the best model’s performance on the test set.

In [23]:
print(grid.best_params_)
print(grid.best_score_)

{'ridge__alpha': 0.155}
0.5277912977828768


In [24]:
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Test set score: 0.51


### 3. Lasso Regression

### Part I

<b>A. Create new features by performing following transformation on inputs: (assume you have named your data frame “sales”) `from math import log, sqrt`</b> 

- `sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)` 
- `sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)`
- `sales['bedrooms_square'] = sales['bedrooms']* sales['bedrooms']`
- `sales['floors_square'] = sales['floors']* sales['floors']`
- Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently, this variable will mostly affect houses with many bedrooms. On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [29]:
from math import log, sqrt
from sklearn.linear_model import Lasso

In [30]:
sales = df.copy()

In [31]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)

In [32]:
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)

In [33]:
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

In [34]:
sales['floors_square'] = sales['floors']*sales['floors']

<b>B. Split the data into train-validate set (90%) and test set (10%) using random state = 0.

In [35]:
X = sales[['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 
           'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 
           'floors_square', 'waterfront', 'view', 'condition', 
           'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']]
y = sales.price

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

<b>C. Run Lasso regression with λ = 100 using the following features on the train-validate set. Remember to run standard scaler to normalize the features before building your model.</b>
- `['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']`

In [42]:
pipe_lasso = make_pipeline(StandardScaler(), Lasso(alpha = 100,max_iter = 30000))

In [43]:
lasso = pipe_lasso.fit(X_train, y_train)

<b>D. Which features have been chosen by LASSO, i.e., which features were assigned nonzero weights? Report the model’s performance on the test set.

In [46]:
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))
print("Number of features used:", np.sum(lasso['lasso'].coef_ != 0))

Test set score: 0.67
Number of features used: 16


<i><b>Features with nonzero weights</i></b>

In [47]:
X_train.columns[(lasso['lasso'].coef_) != 0] 

Index(['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living',
       'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors',
       'floors_square', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'yr_built', 'yr_renovated'],
      dtype='object')

### Part 2 (continue from Part 1)

<b>A. For each L1_penalty λ in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7], use GridSearchCV and 10-fold cross validation to compare the performance of the lasso regression using all the features used in Part 1 on the train-validation set. Remember to run standard scaler to normalize the features.

In [48]:
pipeLas = make_pipeline(StandardScaler(), Lasso())

In [50]:
param_grid = {'lasso__alpha': np.logspace(1, 7, num = 13),
             'lasso__max_iter': [10000]}

In [51]:
grid = GridSearchCV(pipeLas, param_grid = param_grid, cv = 10, n_jobs = -1)

In [52]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lasso',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             ii..._jobs=-1,
             param_gri

<b>B. Report which L1 penalty λ produced the lowest average validation error. Which features have been chosen by the best model, i.e. which features were assigned nonzero weights? Report the best model’s performance on the test set.

In [53]:
print("Score for the best model: {:.2f}".format(grid.score(X_test, y_test)))

Score for the best model: 0.67


In [54]:
grid.best_params_

{'lasso__alpha': 316.22776601683796, 'lasso__max_iter': 10000}

In [55]:
lasso2 = grid.best_estimator_

In [56]:
print(X_train.shape)
np.sum(lasso2['lasso'].coef_ != 0)

(19451, 17)


16

<i><b>Features with nonzero weights</i></b>

In [57]:
X_train.columns[(lasso['lasso'].coef_) != 0] 

Index(['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living',
       'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors',
       'floors_square', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'yr_built', 'yr_renovated'],
      dtype='object')